In [1]:
import pandas as pd
from py2neo import Graph, Node, Relationship
from tqdm import tqdm

In [43]:
df = pd.read_excel('./data/triples_图情学科-传播学.xlsx')
df.head()

,实体1,实体1类型,关系,实体2,实体2类型,学科
0,短视频广告对少数民族地区的传播与影响——基于理塘的调查,研究问题,采用,DPSIR模型,方法模型,2
1,短视频广告对少数民族地区的传播与影响——基于理塘的调查,研究问题,采用,内容分析法,方法模型,2
2,短视频广告对少数民族地区的传播与影响——基于理塘的调查,研究问题,采用,文本分析方法,方法模型,2
3,短视频广告对少数民族地区的传播与影响——基于理塘的调查,研究问题,采用,网络调研,方法模型,2
4,短视频广告对少数民族地区的传播与影响——基于理塘的调查,研究问题,采用,具新能力,方法模型,2


In [54]:
unique_values = df['学科'].unique()
print(unique_values)

[2 1]


In [55]:
row_index = 4
if df.loc[row_index, '学科'] == 1:
    print("第5行的学科为1")
else:
    print("第5行的学科不为1")

第5行的学科不为1


In [51]:
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='创建关系'):
    print(type(row['学科']))
    print(row['学科']==1)

创建关系:  97%|█████████▋| 1155/1195 [00:00<00:00, 5831.33it/s]

<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False


创建关系: 100%|██████████| 1195/1195 [00:00<00:00, 5718.19it/s]

<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
False
<class 'int'>
False
<class 'int'>
False
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True
<class 'int'>
True


In [3]:
g = Graph("http://localhost:7474", name='neo4j', auth=("neo4j", "12345678"))

In [56]:
g.delete_all()

In [55]:
tx = g.begin()

In [7]:
if tx.closed:
    print("事务已关闭")
else:
    print("事务处于打开状态")

# 关闭事务
# tx.commit()

事务处于打开状态


In [59]:
# 单学科知识图谱构建
g_nodes={}
tx = g.begin()
# 循环创建关系和节点
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='创建关系'):
    head_key = '%s-%s' % (row['实体1'], row['实体1类型'])
    if head_key not in g_nodes:
        g_nodes[head_key] = Node(row['实体1类型'], name=row['实体1'])
        tx.create(g_nodes[head_key])
    
    tail_key = '%s-%s' % (row['实体2'], row['实体2类型'])
    if tail_key not in g_nodes:
        g_nodes[tail_key] = Node(row['实体2类型'], name=row['实体2'])
        tx.create(g_nodes[tail_key])
    
    relation = Relationship(g_nodes[head_key], row['关系'], g_nodes[tail_key])
    tx.create(relation)
g.commit(tx=tx)

创建关系:   0%|          | 0/1607 [00:00<?, ?it/s]

创建关系: 100%|██████████| 1607/1607 [00:13<00:00, 115.77it/s]


In [57]:
# 融合知识图谱构建
g_nodes = {}
tx = g.begin()

# 循环创建关系和节点
xueke = Node('学科', name='1')
xueke2 = Node('学科', name='2')

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='创建关系'):
    head_key = '%s-%s' % (row['实体1'], row['实体1类型'])
    if head_key not in g_nodes:
        g_nodes[head_key] = Node(row['实体1类型'], name=row['实体1'])
        tx.create(g_nodes[head_key])
        if row['学科'] == 1:
            # print('yes')
            relation = Relationship(xueke, '包括', g_nodes[head_key])
            tx.create(relation)
        else:
            # print('no')
            relation = Relationship(xueke2, '包括', g_nodes[head_key])
            tx.create(relation)
    
    tail_key = '%s-%s' % (row['实体2'], row['实体2类型'])
    if tail_key not in g_nodes:
        g_nodes[tail_key] = Node(row['实体2类型'], name=row['实体2'])
        tx.create(g_nodes[tail_key])
        if row['学科'] == 1:
            relation = Relationship(xueke, '包括', g_nodes[tail_key])
            tx.create(relation)
        else:
            relation = Relationship(xueke2, '包括', g_nodes[tail_key])
            tx.create(relation)
            
    relation = Relationship(g_nodes[head_key], row['关系'], g_nodes[tail_key])
    tx.create(relation)

g.commit(tx=tx)

创建关系: 100%|██████████| 1195/1195 [00:05<00:00, 213.16it/s]
